In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
# IBMQ.delete_account()
IBMQ.save_account('184ffbf3faf19fd5d79c4a196403b71e67906173e556ce13b7063b68351e2e637ebfa8e5099c52fa0cff26166457db387390c9c00e291d7acd8de779934ecc54', overwrite=True)
IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
from qiskit import QuantumRegister, ClassicalRegister

from qiskit.test.mock import FakeMontreal
device_backend = FakeMontreal()

from qiskit.providers.aer import AerSimulator
sim_Montreal = AerSimulator.from_backend(device_backend)

from qiskit.transpiler import CouplingMap, Layout
from qiskit.transpiler.passes import BasicSwap, LookaheadSwap, StochasticSwap

In [3]:
repetitions = 3

dq = QuantumRegister(3,'data_qubit')
aq = QuantumRegister(2,'ancilla_qubit')
cq = QuantumRegister(1,'control_qubit')
sb = ClassicalRegister(2,'syndrome_bit')
cb = ClassicalRegister(3,'classical_bit')
ccb = ClassicalRegister(1,'classical_control_bit')
qc = QuantumCircuit(dq,aq,cq,sb,cb,ccb)
qc.cx(dq[0],dq[1])
qc.cx(dq[1],dq[2])



for i in range(repetitions):
    
    qc.barrier()
    qc.cx(dq[0],aq[0])
    qc.cx(dq[1],aq[0])
    qc.cx(dq[1],aq[1])
    qc.cx(dq[2],aq[1])
    qc.barrier()
    
    qc.measure(aq[0],sb[0])
    qc.measure(aq[1],sb[1])
    qc.x(dq[0]).c_if(sb, 1)
    qc.x(dq[1]).c_if(sb, 3)
    qc.x(dq[2]).c_if(sb, 2)
    qc.reset(aq[0])
    qc.reset(aq[1])
    



In [4]:
qc.barrier()
qc.measure(dq[0],cb[0])
qc.measure(dq[1],cb[1])
qc.measure(dq[2],cb[2])

qc.measure(cq[0],ccb[0])
qc.draw()

In [11]:
# Transpile the circuit for the noisy basis gates
tcirc = transpile(qc, sim_Montreal)
shots = 1024
# Execute noisy simulation and get counts
result_noise = sim_Montreal.run(tcirc, shots=shots).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise,
               title="[[3,1,3]] bit_flip_error_correction_code #3 cycle")

In [14]:
correct_count = 0
for key in counts_noise.keys():
    data = key[2:5]
    if data == '000':
        correct_count += counts_noise[key]



In [15]:
correct_count

900